In [1]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    # os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
    os.environ["GROQ_API_KEY"] = "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"

In [2]:
from langchain_groq import ChatGroq

In [33]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [6]:
llm_json_mode = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    response_format={"type": "json_object"},
)

c:\Users\Makara\.conda\envs\ai-agents-env\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [44]:
messages = [
    (
        "system",
        "You are a special kind of agent that when i give you an question, you will get answer for that question by solve that in step-by-step.",
    ),
    ("human", "How many `r` letters in the word `strawberry`"),
]
ai_msg = llm.invoke(messages)

In [45]:
ai_msg

AIMessage(content="To find the number of `r` letters in the word `strawberry`, let's break it down step by step:\n\n1. **Write down the word**: The word is `strawberry`.\n2. **Separate the letters**: Separate each letter in the word to count them individually: `s-t-r-a-w-b-e-r-r-y`.\n3. **Count the `r` letters**: Now, count how many times the letter `r` appears in the separated letters:\n   - The first `r` is after `t`.\n   - The second `r` is after `e`.\n   - The third `r` is after the second `r`.\n4. **Conclusion**: There are **3** `r` letters in the word `strawberry`.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 79, 'total_tokens': 240, 'completion_time': 0.769992708, 'prompt_time': 0.018151553, 'queue_time': 0.0005226740000000007, 'total_time': 0.788144261}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c44f7a7-1d08-4ebe-8951-686c2a840fe8-0

In [46]:
from IPython.display import display, Markdown

# Display the response
display(Markdown(ai_msg.content))

To find the number of `r` letters in the word `strawberry`, let's break it down step by step:

1. **Write down the word**: The word is `strawberry`.
2. **Separate the letters**: Separate each letter in the word to count them individually: `s-t-r-a-w-b-e-r-r-y`.
3. **Count the `r` letters**: Now, count how many times the letter `r` appears in the separated letters:
   - The first `r` is after `t`.
   - The second `r` is after `e`.
   - The third `r` is after the second `r`.
4. **Conclusion**: There are **3** `r` letters in the word `strawberry`.

### Search


In [21]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}")
        
_set_env("TAVILY_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "true"

### Tracing


In [72]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "self-correcting-rag-system"

### Vectorestore


In [74]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

In [75]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [79]:
# load documents
docs = [WebBaseLoader(url).load() for url in urls]
doc_list = [item for sublist in docs for item in sublist]

In [83]:
# split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200
)
doc_splits = text_splitter.split_documents(doc_list)

In [ ]:
# add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(
        model="nomic-embed-text-v1",
        inference_mode="local"
    )
)

Downloading: 100%|██████████| 274M/274M [00:57<00:00, 4.80MiB/s] 
Verifying: 100%|██████████| 274M/274M [00:01<00:00, 162MiB/s]  


### Components


#### Router


In [3]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

In [4]:
# prompt
router_instructions = """
    You are an expert at routing a user question to a vectorstore or web search.
    
    The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
    
    Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.
    
    Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question.
"""

In [18]:
# test router
test_web_search = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [
        HumanMessage(
            content="Who is favored to win the NFC Championship game in the 2024 season?"
        )
    ]
)

test_web_search_2 = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="What are the models released today for llama 3.2?")]
)

test_vector_store = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="What are the types of agent memory?")]
)

In [20]:
print(json.loads(test_web_search.content))
print(json.loads(test_web_search_2.content))
print(json.loads(test_vector_store.content))

{'datasource': 'websearch'}
{'datasource': 'websearch'}
{'datasource': 'vectorstore'}
